In [127]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import numpy as np
import tensorflow as tf
import h5py
import pandas as pd
import seaborn as sns
from scipy import stats

from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D, Reshape, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.utils import to_categorical

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [128]:
def get_class_data(class_name):
    global class_counts_test
    
    path_test = os.path.join(test_path, class_name)
    class_units_test = np.fromfile(path_test, dtype=int, count=-1, sep=' ', offset=0)
    units_test = np.array_split(class_units_test, class_units_test.size // 1080, 0) # 1080 = 3 sec * 360Hz
    class_counts_test[class_name] = class_units_test.size // 1080
    
    return units_test

In [129]:
def feature_normalize(dataset):

    mu = np.mean(dataset)
    sigma = np.std(dataset)
    
    return (dataset - mu) / sigma

In [140]:
base_path = 'C:\\Users\\Professional\\Desktop\\1studing\\CourseWork3\\course_work_app\\Course_work\\cnn_code_4_classes'
test_path = os.path.join(base_path, 'data_files_test_4')
model_save_path = os.path.join(base_path, 'model\\cnn_model.h5')
model_weights_path = os.path.join(base_path, 'model\\cnn_model_weights.hdf5')

classes = ['N', 'RBBB', 'B', 'AFIB']
class_mapping = {'N': 0, 'RBBB': 1, 'B': 2, 'AFIB': 3}
class_counts_test = {'N': 0, 'RBBB': 0, 'B': 0, 'AFIB': 0}

X_test = []
Y_test = []
XXXX_test = []
YYYY_test = []

for class_i in classes:
    class_units_test = get_class_data(class_i)
    Y_test  += [class_mapping[class_i]] * class_counts_test[class_i]
    X_test += class_units_test

    XX_temp = []
    XX_temp = np.asarray(X_test)

    YY_temp = []
    YY_temp = np.asarray(Y_test)

    XXX_temp = []
    YYY_temp = []

    XXX_temp = np.array(XXX_temp)
    YYY_temp = np.array(YYY_temp)
    if class_i == 'RBBB':
        XXX_temp = XX_temp[:-2]
        YYY_temp = YY_temp[:-2]
    else:
        if class_i == 'AFIB':
            XXX_temp = XX_temp[:-1]
            YYY_temp = YY_temp[:-1]
        else:
            XXX_temp = XX_temp
            YYY_temp = YY_temp

    X_temp = []
    X_temp = XXX_temp.tolist()
    XXXX_test += X_temp
    Y_temp = []
    Y_temp = YYY_temp.tolist()
    YYYY_test += Y_temp


XXXX_test = np.asarray(XXXX_test) 
YYYY_test = np.asarray(YYYY_test)
XX_test = XXXX_test.astype("float32")
YY_test = YYYY_test.astype("float32")

#print(len(XX_test)) # 5216
#print(len(YY_test))

XXX_test = feature_normalize(XX_test)

num_classes = 4
YYY_test = to_categorical(YY_test, num_classes)


In [141]:
model_test = load_model(model_save_path)
model_test.load_weights(model_weights_path) 

BATCH_SIZE = 32

score = model_test.evaluate(XXX_test, YYY_test, batch_size = BATCH_SIZE, verbose=2)

print("\nAccuracy on test data: %0.2f" % score[1])
print("\nLoss on test data: %0.2f" % score[0])

5216/5216 - 15s - loss: 4.5484 - accuracy: 0.5318

Accuracy on test data: 0.53

Loss on test data: 4.55
